In [9]:
!pip install BeautifulSoup4

distributed 1.21.8 requires msgpack, which is not installed.


In [10]:
!pip install lxml

distributed 1.21.8 requires msgpack, which is not installed.


In [1]:
from bs4 import BeautifulSoup
import csv
import requests
import lxml
from datetime import datetime, timedelta

In [2]:
RU_MONTH_VALUES = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12',
}


def int_value_from_ru_month(date_str):
    for k, v in RU_MONTH_VALUES.items():
        date_str = date_str.replace(k, str(v))
    return date_str

In [3]:
days = timedelta(300)
deadline = datetime.now()-days
    
def get_html(url):
    r = requests.get(url)
    return r.text
 
def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find('div', class_='box-articles')
    fonts = divs.find_all('font', class_='text')[1]
    pages = fonts.find_all('a')[-1].get('href')
    total_pages = pages.split('=')[1].split('&')[0]
    return int(total_pages)
 
#  def write_csv(data):
#     with open('avito.csv', 'a') as f:
#         writer = csv.writer(f)
#         writer.writerow((data['title'],
#                          data['price'],
#                          data['metro'],
#                          data['url']))
 
 
def get_page_data(html):
    
    soup = BeautifulSoup(html, 'lxml')
    news = soup.find_all('div', class_='item')
    for new in news:
        name = new.find('div', class_='iname').getText() 
        url = "https://gisp.gov.ru" + new.find('div', class_='iname').find('a').get('href')
        date_str = new.find('div', class_='date').getText()
        date = int_value_from_ru_month(date_str.lower())
        
        date = datetime.strptime(date, '%d %m %Y')
        if date > deadline:
#             print(date, deadline)
            if "конкурс" in name.lower():
                data = {'name':name,
                    'url':url,
                    'date': date_str}
                print(data)
        else:
            return False
    return True
 
 
def main():
    base_url = "https://gisp.gov.ru/news/"
    page_part = "?PAGEN_1="
 
    
    total_pages = get_total_pages(get_html(base_url))
    fl = True
    for i in range(1, total_pages):
        url_gen = base_url + page_part + str(i)
        html = get_html(url_gen)
        fl = get_page_data(html)
        if fl==False:
            break
#         print(i)

 
if __name__ == '__main__':
    main()

{'name': 'Приглашаем к участию в Первой промышленной премии OEE Award! Впервые в России проходит конкурс проектов по цифровизации предприятий – премия OEE Award. Премия является частью Промышленного марафона «Эффективное производство 4.0».', 'url': 'https://gisp.gov.ru/news/8997721/', 'date': '21 Июня 2018'}
{'name': 'УТВЕРЖДЕНЫ НОВЫЕ ПРАВИЛА ВСЕРОССИЙСКОГО КОНКУРСА "ОРГАНИЗАЦИЯ ОПК ВЫСОКОЙ СОЦИАЛЬНО-ЭКОНОМИЧЕСКОЙ ЭФФЕКТИВНОСТИ"', 'url': 'https://gisp.gov.ru/news/8307243/', 'date': '24 Октября 2017'}
{'name': 'МИНПРОМТОРГ РОССИИ ОБЪЯВЛЯЕТ О НАЧАЛЕ КОНКУРСНОГО ОТБОРА НА ПРАВО ПОЛУЧЕНИЯ СУБСИДИЙ ОРГАНИЗАЦИЯМИ РЕАБИЛИТАЦИОННОЙ ИНДУСТРИИ', 'url': 'https://gisp.gov.ru/news/8292918/', 'date': '5 Октября 2017'}
